### steps
1. Create an sample SQL database
2. create config.ini for Mysql database connection
3. create table
4. performing SQL window on database to solve department highest salary


In [2]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [3]:
def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database


def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """
    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(query_insert, table_values):
    """execute query to insert values to table
    :query_table:: to create scheme table
    :table_values:: to be inserted table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('Inserting values to designed table...')
        cursor.executemany(query_insert, table_values)

        conn.commit()
        print('Values inserted to designeed table')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

In [4]:
def main():
    query_create_employee = "CREATE TABLE Employee (Id INT, \
                                                    Name VARCHAR(255), \
                                                    Salary INT, \
                                                    DepartmentId INT, \
                                                    PRIMARY KEY (Id), \
                                                    FOREIGN KEY (DepartmentId) REFERENCES Department(Id) \
                                                    );"
    
    query_create_department = "CREATE TABLE Department (Id INT, \
                                                        Name VARCHAR(255), \
                                                        PRIMARY KEY (Id) \
                                                        );"
    
    query_insert_employee = "INSERT INTO Employee (Id,Name,Salary,DepartmentId) VALUES(%s,%s,%s,%s)"
    query_insert_department = "INSERT INTO Department (Id,Name) VALUES(%s,%s)"
    ###############
    employee = [(1, 'Joe', 70000, 1), \
                (2, 'Jim', 90000, 1), \
                (3, 'Henry', 80000, 2), \
                (4, 'Sam', 60000, 2), \
                (5, 'Max', 90000, 1)
                ]
    
    department = [(1, 'IT'),
                (2, "Sales")
                ]
    
    create_msql_table(query_create_department)
    create_msql_table(query_create_employee)
    insert_msql_table(query_insert_department, department)
    insert_msql_table(query_insert_employee, employee)

if __name__ == '__main__':
    main()

Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1050 (42S01): Table 'department' already exists
Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1050 (42S01): Table 'employee' already exists
Connecting to MYSQL database...
Connected to MYSQL database
Inserting values to designed table...
Error:  1062 (23000): Duplicate entry '1' for key 'department.PRIMARY'
Connecting to MYSQL database...
Connected to MYSQL database
Inserting values to designed table...
Error:  1062 (23000): Duplicate entry '1' for key 'employee.PRIMARY'


In [30]:
query_fetching_consecutive_number = " \
                                    select RANK() OVER (PARTITION BY Department ORDER BY Salary DESC) as dep_rank, \
                                            Department, \
                                            EmploymentId, \
                                            Employment, \
                                            Salary, \
                                            DepId \
                                    from ( \
                                            select E.Id as EmploymentId, E.Name as Employment, E.Salary, E.departmentId, D.Id as DepId, D.Name as Department\
                                            from Employee E, Department D \
                                            where E.DepartmentId = D.Id \
                                            ) r \
                                    "

try:
    dbconfig = read_db_config()
    conn = MySQLConnection(**dbconfig)
    cursor = conn.cursor()
    cursor.execute(query_fetching_consecutive_number)

    row = cursor.fetchone()

    while row is not None:
        print(row)
        row = cursor.fetchone()

except Error as e:
    print(e)

finally:
    cursor.close()
    conn.close()

(1, 'IT', 5, 'Max', 90000, 1)
(1, 'IT', 2, 'Jim', 90000, 1)
(3, 'IT', 1, 'Joe', 70000, 1)
(1, 'Sales', 3, 'Henry', 80000, 2)
(2, 'Sales', 4, 'Sam', 60000, 2)


In [31]:
!mysql -u root --password=root python_mysql -e "{query_fetching_consecutive_number}"

mysql: [Warning] Using a password on the command line interface can be insecure.
+----------+------------+--------------+------------+--------+-------+
| dep_rank | Department | EmploymentId | Employment | Salary | DepId |
+----------+------------+--------------+------------+--------+-------+
|        1 | IT         |            5 | Max        |  90000 |     1 |
|        1 | IT         |            2 | Jim        |  90000 |     1 |
|        3 | IT         |            1 | Joe        |  70000 |     1 |
|        1 | Sales      |            3 | Henry      |  80000 |     2 |
|        2 | Sales      |            4 | Sam        |  60000 |     2 |
+----------+------------+--------------+------------+--------+-------+
